In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from Functions import seconds

In [ ]:
MagicNumber = "150000"

In [5]:
#-----------------------------------------------------------------
# read in Indicator data into dataframe
#-----------------------------------------------------------------
def read_indicator_data(head=False):
    dataframe = pd.read_csv('data/' + MagicNumber + '_Indicators.csv')
    dataframe.rename(columns={'Date': 'DateTime'}, inplace=True)      # rename 'Date' column to 'DateTime'
    dataframe['DateTime'] = pd.to_datetime(dataframe['DateTime'])     # convert 'DateTime' to pd.datetime
    dataframe['Seconds'] = list(map(seconds, dataframe['DateTime']))  # calc seconds

    #split apart datetime into date and time columns
    #temp = pd.DatetimeIndex(dataframe['DateTime'])
    #dataframe['Date'] = temp.date
    #dataframe['Time'] = temp.time

    cols = dataframe.columns.tolist()           # rearrange to put 'Date' and 'Time' up front behind 'DateTime'
    cols = cols[0:1] + cols[-1:] + cols[1:-1]   #
    dataframe = dataframe[cols]                 #

    dataframe['DateTime'] = pd.to_datetime(dataframe['DateTime'])
    if head:
        print(dataframe.head(head))    
    return dataframe

#-----------------------------------------------------------------    
# Add Rolling stats to dataframe
#-----------------------------------------------------------------
def add_rolling_stats(dataframe, fields, window=15, head=False):
    # TODO - add ability to receive list of fields to apply rolling stats to, then for field in list apply stats
    # TODO - add keyword args to determine which stats to apply
    for field in fields:
        dataframe['Roll_Profit_Count'] = pd.rolling_apply(dataframe[field], window, rolling_profit_count, 1)
        dataframe['Roll_Mean'] = pd.rolling_mean(dataframe[field], window)
        dataframe['Roll_std'] = pd.rolling_std(dataframe[field], window)
        dataframe['Roll_var'] = pd.rolling_var(dataframe[field], window)
    # Order Dataframe with Trends added
    if head:  # print head if passed value
        dataframe[['DateTime', 'Ticket', 'Profit', 'Trend', 'Roll_Profit_Count', 'Roll_Mean', 'Roll_std']].head(head)
    return dataframe

In [3]:
print('Reading Indicator data')
indicator = read_indicator_data()

Reading Indicator data


In [14]:
st = list(indicator.columns[4:])
st

['DOCR(10)',
 'DOCT(10)',
 'DOCS(10)',
 'DOCR(25)',
 'DOCT(25)',
 'DOCS(25)',
 'DOCR(50)',
 'DOCT(50)',
 'DOCS(50)',
 'SMA(25)',
 'SMA(50)',
 'SMA(100)',
 'EMA(25)',
 'EMA(50)',
 'EMA(100)',
 'MACD_MAIN',
 'MACD_SIGNAL',
 'STD(50)',
 'ATR(25)',
 'MOM(25)',
 'RSI(7)',
 'RSI(14)',
 'RSI(30)']

In [15]:
indicator.columns

Index(['DateTime', 'Seconds', 'Ticket', 'Open/Close', 'DOCR(10)', 'DOCT(10)',
       'DOCS(10)', 'DOCR(25)', 'DOCT(25)', 'DOCS(25)', 'DOCR(50)', 'DOCT(50)',
       'DOCS(50)', 'SMA(25)', 'SMA(50)', 'SMA(100)', 'EMA(25)', 'EMA(50)',
       'EMA(100)', 'MACD_MAIN', 'MACD_SIGNAL', 'STD(50)', 'ATR(25)', 'MOM(25)',
       'RSI(7)', 'RSI(14)', 'RSI(30)'],
      dtype='object')

In [21]:
indicator.groupby('Open/Close').count()

,DateTime,Seconds,Ticket,DOCR(10),DOCT(10),DOCS(10),DOCR(25),DOCT(25),DOCS(25),DOCR(50),...,EMA(50),EMA(100),MACD_MAIN,MACD_SIGNAL,STD(50),ATR(25),MOM(25),RSI(7),RSI(14),RSI(30)
Open/Close,,,,,,,,,,,,,,,,,,,,,
ON_NewBar,366111,366111,366111,366111,366111,366111,366111,366111,366111,366111,...,366111,366111,366111,366111,366111,366111,366111,366111,366111,366111


In [22]:
?map